In [1]:
root="C:/github/2023/RatDeconvolution"
path_package="C:/github/enan"

## Preparation of rat gene ontology data

In [2]:
filein = f"{root}/data/rgd.gaf" # from the web site: www.geneontology.org
datafile = f'{root}/data/220801_go.owl' # from the web site: www.geneontology.org
depth=5 # gene ontology depth for enrichment analysis

In [3]:
import csv
from collections import deque, defaultdict

import numpy as np
import pandas as pd


import owlready2
from tqdm import tqdm

In [4]:
class OntologyStructure:
    def __init__(self, datafile, root_term:str):
        """
        constructor
        datafile : str, the url of ontology file
        """
        
        # load datafile
        self.onto = owlready2.get_ontology(datafile).load() # load ontology file (DL from https://www.ebi.ac.uk/efo/)
        print(datafile,"was loaded.")
        
        # determine the roots
        roots = []
        for efo in self.onto.classes(): # EFO0000001 is the root
            if root_term in str(efo):
                roots.append(efo)
        if len(roots) == 0:
            print('the designated root term couldnt be found')
            for efo in self.onto.classed():
                if 'EFO0000001' in str(efo):
                    roots.append(efo)

        # select the first root CAUTION : this must be changed if you use an ontology structure which have multiple roots
        self.root = roots[0] 
        print(self.root, "was determined as root.")
        # with BFS, determine the depth of each GO
        self.min_d = self.__BFS() # 幅優先探索をやってる
        print("Graph structure was obtained.")
        
        self.string_hash = self.__create_string_hash()
        print("String hashtable was created.")
 
    def __BFS(self):
        """
        breadth-first search function
        
        the function to determine the depth of each term
        """
        Q = deque([self.root])
        inf_value = lambda : float('inf')
        min_d = defaultdict(inf_value) # defaultdictでエラーしたらinfを返すようにしている. つまり届かないところ
        min_d[self.root] = 0 # rootの深さは0と定義
        while len(Q)>0:
            v = Q.pop()
            for v1 in self.onto.get_children_of(v): # vの子v1をループ
                if min_d[v1]>10**27: # infより大きい場合. Noneでもいいような. 要するにまだmin_dが定義されていないvの子供が現れた時の判定
                    min_d[v1] = min_d[v] + 1 # min_dがない子供が現れたらvの深さに1段階加えて更新する
                    Q.appendleft(v1) # Qは探索すべきものなので, 新たな階層の探索のために追加する
        return min_d # 木構造を返す: タームを入れるとその深さを返すもの

    def __create_string_hash(self):
        """
        string hash function
        
        create the hashtable to get the class instance corresponding to the given string
        """
        
        string_hash = defaultdict(str)
        for key in self.min_d:
            string_hash[str(key)] = key
        return string_hash

    def get_num(self, depth):
        """
        getting the number of ontology term in a specific level
        depth : int
        """
        count = 0
        for key in self.min_d:
            if self.min_d[key]==depth:
                count+=1
        return count
    
    def get_term(self, depth):
        """
        getting the term in the specific level
        depth :  int
        """
        res = []
        term = []
        for key in self.min_d:
            if self.min_d[key]==depth:
                res.append(str(key))
                term.append(str(key.label[0]))
        return res,term
    
    def get_downstream_ID(self):
        max_depth = max(list(self.min_d.values()))
        print('max depth :',max_depth)
        total_res = []
        total_term = []
        for i in range(1,max_depth+1):
            tmp_res = []
            tmp_term = []
            for key in self.min_d:
                if self.min_d[key]==i:
                    tmp_res.append(str(key))
                    tmp_term.append(str(key.label[0]))
            
            for t in tmp_res:
                total_res.append(t)
            for s in tmp_term:
                total_term.append(s)
        return total_res, total_term
    
    def get_upstream(self, go, depth):
        """
        getting the upstream GO in a specific level
        
        go : GO class instance
        depth : int
        """
        if self.min_d[go]<2:
            return set()
        if self.min_d[go]==2:
            return set([go])
        
        up = set()

        Q = deque([go])
        while len(Q)>0:
            v = Q.pop()
            try:
                for v1 in self.onto.get_parents_of(v):
                    if self.min_d[v1]==depth:
                        up.add(v1)
                    else:
                        Q.appendleft(v1)
            except:
                pass
            
        return up
    
    def str2go(self, string):
        """
        str to GO class instance
        """
        return self.string_hash[string]

    def go2str(self, go):
        """
        GO class instance to str
        """
        return str(go)

    def get_depth(self, go):
        """
        get the depth of GO
        """
        return self.min_d[go]
        
    def get_term(self, depth):
        """
        getting the term in the specific level
        depth :  int
        """
        res = []
        term = []
        for key in self.min_d:
            if self.min_d[key]==depth:
                res.append(str(key))
                term.append(str(key.label[0]))
        return res,term
    
    def get_downstream(self, go):
        """
        getting the deepest downstream go terms
        go : GO class instancce
        """
        res = []
        Q = deque(self.onto.get_children_of(go))
        if len(Q)==0:
            return [go]
        
        while len(Q)>0:
            v = Q.pop()
            res.append(v)
            v_child = self.onto.get_children_of(v)
            if len(v_child)==0:
                pass
            else:
                for v1 in v_child:
                    Q.appendleft(v1)
        res = list(set(res))
        return res     

def load_go(gene_dict=dict()):
    terms = []
    members = []
    for key in gene_dict.keys():
        terms.append(key)
        members.append(gene_dict[key])
    return terms, members

def create_dict(datafile='', gene_dict=dict(), depth:int=4):
    # load go.owl file
    dat = OntologyStructure(datafile, 'obo.GO_0008150') #GO_0008150 : Biological Process
    res, term = dat.get_term(depth)
    print('Indicated depth GO terms : {}'.format(len(res)))
    corresp_res = [dat.get_downstream(dat.str2go(i)) for i in tqdm(res)]
    corresp_res = [[str(v).replace('obo.GO_', 'GO:') for v in i] for i in corresp_res]

    # load gene file
    terms, members = load_go(gene_dict=gene_dict)
    #terms = [i.split('(')[1].split(')')[0] for i in terms]
    go_dict = dict(zip(terms, members))
    
    # create dict
    res_dict=dict()
    goperterm=[]
    for root, i, t in zip(res,corresp_res, term):
        res_temp = set()
        for v in i:
            genes_temp=go_dict.get(v, 'no')
            if genes_temp=='no':
                pass
            else:
                res_temp = res_temp or genes_temp
        if len(res_temp)>0:
            res_dict[root.replace('obo.GO_','GO:')+'_'+t]=res_temp
            goperterm.append(len(res_temp))

    print('No. of GO terms : {}'.format(len(res_dict)))
    print('No. of genes / term : {}'.format(np.mean(goperterm)))

    return res_dict

def extract(sentence):
    res = []
    temp1 = sentence.split("RGD\t")
    flag=False
    for i in temp1:
        temp2 = i.split("\t")
        if len(temp2)>5:
            symbol = temp2[1]
            term = temp2[3]
            if "GO:" in term:
                res.append([symbol, term])
                flag=True
    temp1 = sentence.split("UniProtKB\t")
    for i in temp1:
        temp2 = i.split("\t")
        if len(temp2)>5:
            symbol = temp2[1]
            term = temp2[3]
            if "GO:" in term:
                res.append([symbol, term])
                flag=True
    if flag:
        return res, []
    else:
        return res, [sentence]

In [5]:
with open(filein, encoding='utf-8', newline='') as f:
    reader = csv.reader(f)
    res = [cols for cols in reader]

res = res[39:]
res_open=[]
res_error1=[]
res_error2=[]
for i in res:
    if type(i)==str:
        res_temp, error = extract(i)
        res_open += res_temp
        res_error1 += error
    elif len(i)==1:
        res_temp, error = extract(i[0])
        res_open += res_temp
        res_error1 += error
    else:
        for v in i:
            if type(v)==str:
                res_temp, error = extract(v)
                res_open += res_temp
                res_error2 += error
            else:
                res_temp, error = extract(v[0])
                res_open += res_temp
                res_error2 += error
e2_go = []
for i in res_error2:
    if "GO" in i:
        e2_go.append(i)

In [6]:
e2_go[:10]

['regulates(GO:0006366)\t',
 '8-sialyltransferase 5\t\tgene\ttaxon:10116\t20220421\tGOC\t\t',
 '8-sialyltransferase 5\t\tgene\ttaxon:10116\t20220421\tGOC\t\t',
 ' ankyrin repeat and coiled-coil containing 1\t\tgene\ttaxon:10116\t20180711\tSynGO\tpart_of(UBERON:0002421)\t',
 ' ankyrin repeat and coiled-coil containing 1\t\tgene\ttaxon:10116\t20180711\tSynGO\tpart_of(UBERON:0002421)\t',
 ' ankyrin repeat and coiled-coil containing 1\t\tgene\ttaxon:10116\t20180711\tSynGO\tpart_of(GO:0098978)',
 ' ankyrin repeat and coiled-coil containing 1\t\tgene\ttaxon:10116\t20180711\tSynGO\tpart_of(GO:0098978)',
 ' ankyrin repeat and coiled-coil containing 1\t\tgene\ttaxon:10116\t20180711\tSynGO\toccurs_in(GO:0098978)',
 ' ankyrin repeat and coiled-coil containing 1\t\tgene\ttaxon:10116\t20180711\tSynGO\toccurs_in(GO:0098978)',
 ' member 2\t\tgene\ttaxon:10116\t20050726\tRGD\tnegatively_regulates(GO:0008283)\t']

In [7]:
res_dict = dict()
term_set = set()
for gene, term in res_open:
    if not term in term_set:
        term_set.add(term)
        res_dict[term]=set([gene])
    else:
        res_dict[term].add(gene)

In [8]:
res = create_dict(datafile=datafile, gene_dict=res_dict, depth=depth)

C:/github/2023/RatDeconvolution/data/220801_go.owl was loaded.
obo.GO_0008150 was determined as root.
Graph structure was obtained.
String hashtable was created.
Indicated depth GO terms : 6712


100%|██████████| 6712/6712 [00:00<00:00, 8258.26it/s]


No. of GO terms : 3447
No. of genes / term : 9.931534667827096


In [10]:
pd.to_pickle(res, f"{root}/result/go_rat_depth5.pickle")

## ssGSEA

In [13]:
# import
import sys
import warnings

import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import copy
import codecs

sys.path.append("C:/github/enan")
from enan import binom, ssgsea

warnings.simplefilter('ignore')

In [20]:
def annotation_sample(df, df_b):
    temp = df_b.loc[df.columns.tolist()]
    name = temp["COMPOUND_NAME"].tolist()
    dose = temp["DOSE_LEVEL"].tolist()
    time = temp["SACRIFICE_PERIOD"].tolist()
    ind = [f"{i}_{j}_{k}" for i, j, k in zip(name, dose, time)]
    return ind

def load_transcriptome(target_compounds):
    # load transcriptome
    df_target = pd.read_csv(f"{root}/data/tggate_transcriptome.csv",index_col=0)
    df_sample = pd.read_csv(f"{root}/data/tggate_sample_information.csv",index_col=0)
    df_target.columns=[str(i) for i in df_target.columns]
    df_sample.index=[str(i) for i in df_sample.index]
    df_target.columns = annotation_sample(df_target, df_sample)
    target_lst = [
        f'{compound}_{conc}_{time}'
        for compound in target_compounds 
        for conc in ["High", "Control"]
        for time in ["3 hr", "6 hr", "9 hr", "24 hr"]
    ]
    df_target=df_target.loc[:,target_lst]
    return df_target

def calc_ssGSEA(df, depth=5, limit=10):
    # load
    set_depth = pd.read_pickle(f"{root}/result/go_rat_depth{str(depth)}.pickle")
    ref = dict()
    set_whole=set()
    for i in set_depth.keys():
        temp = set_depth[i]
        if len(temp)>limit-1:
            ref[i]=temp
            set_whole = set_whole|set_depth[i]
    print(f"ref length: {len(ref)}")
    print(f"whole genes: {len(set_whole)}")

    dat = ssgsea.ssGSEA()
    dat.fit(ref)
    dat.set_whole(set_whole)
    res = dat.calc(df, method="kuiper")
    return res

In [15]:
target_compounds=[
    "naphthyl isothiocyanate","bromobenzene","simvastatin","enalapril",
    "gefitinib","metformin","tiopronin","colchicine",
    "bortezomib","methylene dianiline","galactosamine","thioacetamide",
    "LPS","cycloheximide","tacrine","nitrofurazone",
    ]

In [18]:
df = load_transcriptome(target_compounds)
col = df.columns.tolist()
df.columns=range(len(df.columns))

In [21]:
res = calc_ssGSEA(df, depth=5)
res.columns=col

ref length: 600
whole genes: 8989
Kuiper method


100%|██████████| 383/383 [06:43<00:00,  1.05s/it]


In [22]:
res.to_csv(f"{root}/result/res_depth5_kuiper.csv")

## Analysis/Visualization